# Test RNN sui dati preprocessati di SEED-IV

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, GRU, Dense, Masking, Bidirectional
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/FVAB/dataset_de_LDS_SEEDIV.csv")
df.drop(['Unnamed: 0'],axis=1, inplace=True)
df = df.set_index(['id_user','session','video'])

In [ ]:
df[df.isnull()==True] = -10  #sostituisce i valori nulli con -10 per applicare la maschera successiva

Metodo per effettuare il reshape dei dataset

In [ ]:
def preprocessing_for_LSTM(X_train, X_test, y_train, y_test):
    X_train = np.array(X_train).reshape((X_train.shape[0], X_train.shape[1], 1))
    y_train = to_categorical(y_train)
    X_test = np.array(X_test).reshape((X_test.shape[0], X_test.shape[1], 1))
    y_test = to_categorical(y_test)
    return X_train, X_test, y_train, y_test

# Subject dependent

In [ ]:
def subject_dependent_split(df,session):
    df = df.reset_index()
    np.random.seed(75)
    test_videos = np.random.choice(np.arange(1, 25), replace=False, size=(8))
    df_sess = df.loc[df['session'] == session]
    X_test = df_sess[df_sess['video'].isin(test_videos)].set_index(['id_user','session','video']).drop('emotion',axis=1)
    y_test = df_sess[df_sess['video'].isin(test_videos)].set_index(['id_user','session','video']).emotion
    X_train = df_sess[~df_sess['video'].isin(test_videos)].set_index(['id_user','session','video']).drop('emotion',axis=1)
    y_train = df_sess[~df_sess['video'].isin(test_videos)].set_index(['id_user','session','video']).emotion
    return X_train, X_test, y_train, y_test

In [ ]:
df

In [ ]:
accuracy_test = []
model = Sequential()
for i in range(1,4):
    X_trainSD, X_testSD, y_trainSD, y_testSD = subject_dependent_split(df,i)
    X_trainSD, X_testSD, y_trainSD, y_testSD = preprocessing_for_LSTM(X_trainSD, X_testSD, y_trainSD, y_testSD)
    del model
    model = Sequential()
    #model.add(Masking(mask_value=-10,input_shape=(X_trainSD.shape[1], 1)))
    model.add(Bidirectional(GRU(units=128, input_shape=(X_trainSD.shape[1], 1))))
    model.add(Dense(units=4, activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_trainSD, y_trainSD, epochs=300, batch_size=64)
    loss, accuracy = model.evaluate(X_testSD, y_testSD)
    accuracy_test.append(accuracy)

In [ ]:
print("ACCURACY: ", np.array(accuracy_test).mean())

In [ ]:
accuracy_test = []
model = Sequential()
for i in range(1,4):
    X_trainSD, X_testSD, y_trainSD, y_testSD = subject_dependent_split(df,i)
    X_trainSD, X_testSD, y_trainSD, y_testSD = preprocessing_for_LSTM(X_trainSD, X_testSD, y_trainSD, y_testSD)
    del model
    model = Sequential()
    #model.add(Masking(mask_value=-10,input_shape=(X_trainSD.shape[1], 1)))
    model.add(LSTM(units=128, input_shape=(X_trainSD.shape[1], 1))))
    model.add(Dense(units=4, activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_trainSD, y_trainSD, epochs=300, batch_size=64)
    loss, accuracy = model.evaluate(X_testSD, y_testSD)
    accuracy_test.append(accuracy)

print("ACCURACY: ", np.array(accuracy_test).mean())

# Subject-Biased Experiment


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('emotion',axis=1), df.emotion, test_size=0.20, random_state=44)
X_train, X_test, y_train, y_test = preprocessing_for_LSTM(X_train, X_test, y_train, y_test)

In [ ]:
del model
model = Sequential()
#model.add(Masking(mask_value=-10,input_shape=(X_train.shape[1], 1)))
model.add(Bidirectional(GRU(units=32, input_shape=(X_train.shape[1], 1))))
model.add(Dense(units=4, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=200, batch_size=16)

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)

In [ ]:
del model
model = Sequential()
#model.add(Masking(mask_value=-10,input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=32, input_shape=(X_train.shape[1], 1))))
model.add(Dense(units=4, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=200, batch_size=16)

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)

# Subject Indipendent

In [ ]:
def subject_independent_split(df,session):
    df = df.reset_index()
    df_sess = df.loc[df['session'] == session]
    groups = df_sess['id_user']
    X = df_sess.set_index(['id_user','session','video']).drop('emotion',axis=1)
    y = df_sess.set_index(['id_user','session','video']).emotion
    return X, y, groups

In [ ]:
X1, y1, groups1 = subject_independent_split(df,1)
X2, y2, groups2 = subject_independent_split(df,2)
X3, y3, groups3 = subject_independent_split(df,3)

In [ ]:
train = [X1, X2, X3]
targets = [y1, y2, y3]
groups = [groups1, groups2, groups3]

In [ ]:
logo = LeaveOneGroupOut()

In [ ]:
accuracy_all = []
for n in range (0,3):
  X = train[n]
  y = targets[n]
  group = groups[n]
  for i, (train_index,test_index) in enumerate(logo.split(X,y,group)):

    X_train = X.iloc[train_index]
    y_train = y.iloc[train_index]
    X_test = X.iloc[test_index]
    y_test = y.iloc[test_index]
    X_train, X_test, y_train , y_test = preprocessing_for_LSTM(X_train,X_test,y_train,y_test)

    model = Sequential()
    #model.add(Masking(mask_value=-10,input_shape=(X_train.shape[1], 1)))
    model.add(Bidirectional(GRU(units=32, input_shape=(X_train.shape[1], 1))))
    model.add(Dense(units=4, activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=200, batch_size=64)
    loss, accuracy = model.evaluate(X_test, y_test)
    accuracy_all.append(accuracy)

In [ ]:
accuracy_all = []
for n in range (0,3):
  X = train[n]
  y = targets[n]
  group = groups[n]
  for i, (train_index,test_index) in enumerate(logo.split(X,y,group)):

    X_train = X.iloc[train_index]
    y_train = y.iloc[train_index]
    X_test = X.iloc[test_index]
    y_test = y.iloc[test_index]
    X_train, X_test, y_train , y_test = preprocessing_for_LSTM(X_train,X_test,y_train,y_test)

    model = Sequential()
    #model.add(Masking(mask_value=-10,input_shape=(X_train.shape[1], 1)))
    model.add(LSTM(units=32, input_shape=(X_train.shape[1], 1))))
    model.add(Dense(units=4, activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=200, batch_size=64)
    loss, accuracy = model.evaluate(X_test, y_test)
    accuracy_all.append(accuracy)